In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset
import numpy as np
import pandas as pd

class Data(Dataset):

    def __init__(self, dt: pd.DataFrame, window_size: int = 20):
        """
        feature:
        "收盘价", "开盘价", "最高价", "最低价", "成交量"
        """
        super().__init__()
        self.values = torch.from_numpy(dt.iloc[:, 2:].values).type(dtype=torch.float32)
        self.window_size = window_size
        self.num_feature = self.values.shape[1]

    def __getitem__(self, idx: int):
        # (N_batch, seq_length, num_feature)
        x = self.values[idx:(idx+self.window_size)].reshape((self.window_size, self.num_feature))
        y = self.values[idx+1:(idx+self.window_size+1), 0].reshape((self.window_size, 1))[-1]
        return x, y

    def __len__(self):
        return self.values.shape[0] - self.window_size

In [3]:
class AccRate:
    """
    https://flyai.com/d/StockPredict
    挑战赛上的反向误差率：
    """
    def __init__(self):
        pass

    def __call__(self, output: torch.Tensor, x: torch.Tensor):
        high = x[:, 2]
        low = x[:, 3]
        gt = x[:, 0]
        return 100 * (1 - torch.sum((output - gt) / (high - low))/x.shape[0])
    
acc_rate = AccRate()

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch

def plot(ground_truth: pd.Series, pred: np.ndarray, tag: int, save_file: str, error: float):
    if isinstance(pred, list):
        pred = np.asarray(pred)
    if isinstance(error, torch.Tensor):
        error = error.item()
    if not os.path.exists("output"):
        os.mkdir("output") 
    plt.figure()
    # plt.title(f"result: {error}")
    pre = np.ones_like(pred)
    post = np.ones_like(pred)
    pre[tag:] = pred[tag:] * np.inf
    post[:tag] = post[:tag] * np.inf

    plt.plot(ground_truth.values, label="ground truth", color="green")
    plt.plot(pred * pre, label="predication(train stage)", linestyle="--", color="purple")
    plt.plot(pred * post, label="predication(test stage)", linestyle="--", color="red")
    plt.legend()
    plt.axvline(x=tag, linestyle=":", color="cyan")
    plt.savefig(f"output/{save_file}.png", bbox_inches="tight")
    plt.close()

In [21]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from glob import glob
from typing import cast
import pandas as pd

EPOCHS = 100
WINDOW_SIZE = 40

class GRU(torch.nn.Module):
    def __init__(self, feature_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        # GRU + 全连接
        self.gru = nn.GRU(feature_dim, hidden_dim, layer_dim,
                         batch_first=True)
        self.fc1 = nn.Sequential(
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, inputs):
        x, _ = self.gru(inputs, None)# [batch, time_step, hidden_dim]
        x = self.fc1(x[:, -1, :]) #[batch, time_step, output_dim]
        return x

split_data_root = os.path.join(os.path.dirname(os.path.abspath('__file__')), "split")
# files = os.listdir(split_data_root)
# files = ['000622.SZ.csv']:
files = np.loadtxt("./comp_idx.txt", dtype=str).tolist()
if isinstance(files, str):
    files = [files] 
for c_idx in files:
        # 模型
        model = GRU(feature_dim=5, hidden_dim=20, layer_dim=1, output_dim=1)
        opt = torch.optim.Adam(model.parameters(), lr=1e-4)
        criterion = nn.SmoothL1Loss()
        if torch.cuda.is_available():
            model.cuda()

        # 数据
        csv = glob(os.path.join(split_data_root, f'{c_idx}.csv'))[0]
        dt = pd.read_csv(csv)
        dt[["收盘价","开盘价","最高价","最低价","成交量"]] = (dt[["收盘价","开盘价","最高价","最低价","成交量"]] - dt[["收盘价","开盘价","最高价","最低价","成交量"]].mean()) / dt[["收盘价","开盘价","最高价","最低价","成交量"]].std()
        length = len(dt)
        train_len = round(length * 0.8) 
        train_dt = dt[:train_len]
        test_dt = dt[(train_len-WINDOW_SIZE+1):]

        train_data = Data(train_dt, window_size=WINDOW_SIZE)
        train_data = DataLoader(train_data, batch_size=1, pin_memory=True, num_workers=4)

        test_data = Data(test_dt, window_size=WINDOW_SIZE)
        test_data = DataLoader(test_data, batch_size=1, pin_memory=True, num_workers=4)

        ground_truth = dt[WINDOW_SIZE:len(dt)-1].iloc[:, 2]

        outputs_train = []
        outputs_test = []
        
        # 训练
        import datetime
        starttime = datetime.datetime.now()

        tq = tqdm(range(EPOCHS), total=EPOCHS, desc="Training..")
        for ep in tq:
            for x, y in train_data:
                x, y = cast(torch.Tensor, x), cast(torch.Tensor, y)
                if torch.cuda.is_available():
                    x = x.cuda()
                    y = y.cuda()
                out= model(x)
                if ep == EPOCHS - 1:
                    with torch.no_grad():
                        outputs_train.extend(out.cpu().detach().squeeze(dim=1).numpy().tolist())
                loss = criterion(out, y)
                opt.zero_grad()
                loss.backward()
                opt.step()
                tq.set_postfix({
                    "loss": loss.item()
                })
        endtime = datetime.datetime.now()
        print(f'训练时间：{(endtime - starttime).seconds}')
        # 测试
        test_x = []
        with torch.no_grad():
            for x, y in tqdm(test_data, total=len(test_data), desc="Testing..."):
                x, y = cast(torch.Tensor, x), cast(torch.Tensor, y)
                if torch.cuda.is_available():
                    x = x.cuda()
                    y = y.cuda()
                out = model(x)
                outputs_test.extend(out.cpu().detach().squeeze(dim=1).numpy().tolist())
                test_x.extend(x[:, -1].cpu().detach().numpy().tolist())

    #     er = acc_rate(torch.as_tensor(outputs_test), torch.as_tensor(test_x))
    #     print(f'acc_rate:{er}')
        plot(ground_truth, outputs_train+outputs_test, train_len - WINDOW_SIZE, f"GRU_{c_idx}", error=0.1)

Testing...:   0%|          | 0/47 [00:00<?, ?it/s]

训练时间：98


Testing...:   0%|          | 0/47 [00:00<?, ?it/s]

训练时间：96


Testing...:   0%|          | 0/47 [00:00<?, ?it/s]

...

Testing...:   0%|          | 0/47 [00:00<?, ?it/s]

训练时间：95


Testing...:   0%|          | 0/47 [00:00<?, ?it/s]

训练时间：96


Testing...: 100%|██████████| 47/47 [00:00<00:00, 175.37it/s]


In [11]:
from thop import profile

model = GRU(feature_dim=5, hidden_dim=20, layer_dim=1, output_dim=1)
input = torch.randn(1, 40, 5)
flops, params = profile(model, inputs=(input, ))
print('flops:{}'.format(flops))
print('params:{}'.format(params))

[INFO] Register count_gru() for <class 'torch.nn.modules.rnn.GRU'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class '__main__.GRU'>. Treat it as zero Macs and zero Params.
flops:70420.0
params:1641.0
